In [1]:
import nltk
import numpy as np
from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import treebank
from nltk import sent_tokenize, pos_tag_sents

In [2]:
all_sent = treebank.tagged_sents()
tn = TreebankWordTokenizer()
para = "She is working. How are you doing?"
sentences = sent_tokenize(para)
sent_tokens = []
for sentence in sentences:
    tokenized_sent = tn.tokenize(sentence)
    sent_tokens.append(tokenized_sent)
sent_tagged = pos_tag_sents(sent_tokens)


# trivial_solve (Hacky)
past_tag = ["VBD", "VBN"]
present_tag = ["VBP", "VBZ","VBG"]
future_modal = ["will","ll","shall","might","may",]
past_modal = ["would","could"]

para_word_tense = []
para_tense = []
for sentence in sent_tagged:
    sent_word_tense = []
    sent_tense = []
    for (word, postag) in sentence:
        tense = "UNK"
        if(postag in past_tag):
            tense = "Past"
        elif(postag in present_tag):
            tense = "Present"
        elif(postag == "MD"):
            if(word.lower() in future_modal):
                tense = "Future"
            elif(word.lower() in past_modal):
                tense = "Past"
            else:
                tense = "present"
        sent_word_tense.append((word, tense))
        if(tense!="UNK"):
            sent_tense.append(tense)
    para_word_tense.append(sent_word_tense)
    para_tense.append(sent_tense)
if(len(sent_tense)==0):
        sent_tense.append("UNK")
        para_tense.append(sent_tense)
print(para_tense)

[['Present', 'Present'], ['Present', 'Present']]


In [3]:
'''
 Given a parse tree, find out the main tense of the sentence.
'''
index = int(input("Enter Index[0,3913]: "))
tree = treebank.parsed_sents()[index]
s_tagged = treebank.tagged_sents()[index]
s = ' '.join(treebank.sents()[index])
print("Sentence: ", s)
print("**************************")
print("Tree")
print("**************************")
print(tree)
print("**************************")
print("Tagged sentence")
print("**************************")
print(s_tagged)
print("**************************")

#POS-based tense detection
verb_list = ["VBZ","VBG","VBD","MD","VBN","VBP","VB"]

# Algorithm_1
def getTenseFromVerb(postag, word):
    past_tag = ["VBD", "VBN"]
    present_tag = ["VBP", "VBZ"]
    future_modal = ["will","ll","shall"]
    past_modal = ["would","could"]
    tense = "UNK"
    if(postag in past_tag):
        tense = "Past"
    elif(postag in present_tag):
        tense = "Present"
    elif(postag == "MD"):
        if(word in future_modal):
            tense = "Future"
        elif(word in past_modal):
            tense = "Past"
        else:
            tense = "present"
    return tense

#Algorithm 2 [For finding main tense from parse tree]
def go_deep(node, choice_label):
    tense = "UNK"
    if(type(node) == nltk.tree.Tree):
        for child in node:
            if(type(child) == nltk.tree.Tree and child.label() == choice_label):
                print("Child:")
                tense = getTenseFromVerb(child)
                if(tense!= "UNK"):
                    print("TENSE: ",tense)
                    tenses.append(tense)
                    return (tense)
    return tense

def processing(tree):
    if(type(tree) == nltk.tree.Tree):
        count = 0
        for subtree in tree:
            if(subtree.label() in verb_list):
                return getTenseFromVerb(subtree.label(),subtree.leaves()[count])
            else:
                return None
            count = count + 1

def traverse_tree(tree):
    for subtree in tree:
        if(len(subtree) > 1):
            if(subtree.label() == "VP"):
                main_tense = processing(subtree)
    if(main_tense != None):
        return main_tense
    for subtree in tree:
        if(len(subtree) > 1):
            if(subtree.label() == "S"):
                main_tense = processing(subtree)
    if(main_tense == None):
        main_tense = "UNK"
    return main_tense

main_tense = traverse_tree(tree)
print("Main Tense: ",main_tense)

Enter Index[0,3913]: 0
Sentence:  Pierre Vinken , 61 years old , will join the board as a nonexecutive director Nov. 29 .
**************************
Tree
**************************
(S
  (NP-SBJ
    (NP (NNP Pierre) (NNP Vinken))
    (, ,)
    (ADJP (NP (CD 61) (NNS years)) (JJ old))
    (, ,))
  (VP
    (MD will)
    (VP
      (VB join)
      (NP (DT the) (NN board))
      (PP-CLR (IN as) (NP (DT a) (JJ nonexecutive) (NN director)))
      (NP-TMP (NNP Nov.) (CD 29))))
  (. .))
**************************
Tagged sentence
**************************
[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')]
**************************
Main Tense:  Future
